In [1]:
import pytorch_lightning as pl
import torch
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights, convnext_base, ConvNeXt_Base_Weights, convnext_small, ConvNeXt_Small_Weights, efficientnet_b4, EfficientNet_B4_Weights
import torch.nn as nn
import torch.optim as optim
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from typing import List, Dict, Optional
import pandas as pd
import numpy as np
import os

import albumentations as albu
from albumentations.pytorch import ToTensorV2
import random
import matplotlib.pyplot as plt

from pathlib import Path
import random
import cv2

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class AugmentationTransforms:
    def __init__(self, image_size: int):
        self.image_size = image_size

    def get_training_augmentation(self):
        scale_factor = random.uniform(1.0, 1.05)
        train_transform = [
            albu.HorizontalFlip(p=0.5),
            albu.augmentations.geometric.resize.Resize(
                int(self.image_size * scale_factor),
                int(self.image_size * scale_factor),
                always_apply=True,
            ),
            albu.RandomCrop(
                height=self.image_size, width=self.image_size, always_apply=True
            ),
            albu.augmentations.transforms.GaussNoise(p=0.2),
            albu.augmentations.geometric.transforms.Perspective(p=0.5),
            albu.OneOf(
                [
                    albu.CLAHE(p=1),
                    albu.RandomBrightnessContrast(p=1),
                    albu.RandomGamma(p=1),
                ],
                p=0.5,
            ),
            albu.OneOf(
                [
                    albu.augmentations.transforms.Sharpen(p=1),
                    albu.Blur(blur_limit=3, p=1),
                    albu.MotionBlur(blur_limit=3, p=1),
                ],
                p=0.5,
            ),
            albu.OneOf(
                [albu.RandomBrightnessContrast(p=1), albu.HueSaturationValue(p=1),],
                p=0.5,
            ),
              albu.augmentations.geometric.resize.Resize(
                self.image_size, self.image_size, always_apply=True
            ),
        ]
        return albu.Compose(train_transform)

    def get_validation_augmentation(self):
        """Add paddings to make image shape divisible by 32"""
        test_transform = [
            albu.augmentations.geometric.resize.Resize(
                self.image_size, self.image_size, always_apply=True
            ),
        ]
        return albu.Compose(test_transform)

    def get_preprocessing(self):
        """Construct preprocessing transform

        Args:
            preprocessing_fn (callbale): data normalization function
                (can be specific for each pretrained neural network)
        Return:
            transform: albumentations.Compose

        """

        # Model expects input [N, C, H, W]
        # ToTensor convert HWC image to CHW image
        ubc_mean = [0.8894420586142374,0.8208752169441305,0.8864016141389351]
        ubc_std = [0.10106393015358608,0.15637655015581306,0.09892687853183287]
        transform = [
            albu.Normalize(mean=ubc_mean, std=ubc_std),
            ToTensorV2(),
        ]

        return albu.Compose(transform)


In [3]:
import timm

class EfficientNetMIL(nn.Module):
    def __init__(self, num_classes: int, init_weights: bool):
        super(EfficientNetMIL, self).__init__()
        # Options for output is 0, ..., 4 with following shapes if input is (1, 3,512,512)
        # torch.Size([1, 24, 256, 256])
        # torch.Size([1, 32, 128, 128])
        # torch.Size([1, 56, 64, 64])
        # torch.Size([1, 160, 32, 32])
        # torch.Size([1, 448, 16, 16])
        self.feature_extractor1 = timm.create_model('efficientnet_b4', pretrained=init_weights, features_only=True, out_indices=[3])
        
#         self.classifier = nn.Sequential(
#             nn.Conv2d(448, 1792, kernel_size=(1,1), stride=(1,1), bias=False),
#             nn.BatchNorm2d(1792),
#             nn.SiLU(inplace=True),
#             nn.AdaptiveAvgPool2d(output_size=1),
#             nn.Dropout(p=0.4),
#             nn.Linear(1792, num_classes),
#         )
        self.feature_extractor2 = nn.Sequential(
            nn.Conv2d(160, 56, kernel_size=(3,3), stride=(1,1), bias=False),
            nn.BatchNorm2d(56),
            nn.SiLU(inplace=True),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.4),
            nn.Linear(56 * 14 * 14, num_classes),
        )
        
    def forward(self, x):
        # TIMM returns a list 
        x = self.feature_extractor1(x)[0]
        bag_feature = torch.mean(x, axis=0, keepdims=True)
        bag = self.feature_extractor2(bag_feature)
        bag = bag.view(-1, 56 * 14*14)
        y = self.classifier(bag)
        
        return y

In [4]:
import torchmetrics


class CancerDetector(pl.LightningModule):
    def __init__(
        self,
        lr: float,
        gamma: float,
        model_name: str,
        batch_size: int,
        warmup_epochs: int = 4,
        num_classes: int = 5,
        init_weights: bool = True,
    ):
        super().__init__()
        # TODO Use model preprocessing function
        self.model = self._get_model(model_name, num_classes, init_weights)

        self.loss_fn = nn.CrossEntropyLoss()
        self.lr = lr
        self.gamma = gamma
        self.warmup_epochs = warmup_epochs
        self.batch_size = batch_size

        self.save_hyperparameters()

        # Should we use macro average? Default is micro
        self.accuracy = torchmetrics.classification.Accuracy(
            num_classes=num_classes, task="multiclass"
        )
        self.f1 = torchmetrics.classification.F1Score(
            num_classes=num_classes, task="multiclass"
        )
        self.recall = torchmetrics.classification.Recall(
            num_classes=num_classes, task="multiclass"
        )
        self.precision = torchmetrics.classification.Precision(
            num_classes=num_classes, task="multiclass"
        )

    def _get_model(self, model_name: str, num_classes: int, init_weights: bool):
        if model_name == "efficientnet_b4_mil":
            model = EfficientNetMIL(num_classes, init_weights)
        else:
            raise Exception(f"Unknown model name {model_name}")

        return model

    def forward(self, imgs: torch.Tensor):
        return self.model(imgs)

    def training_step(self, batch: torch.Tensor, batch_idx: int):
        x, y = batch
        output = self(x)
        loss = self.loss_fn(output, y)

        self._log_metrics(loss, output, y, "train")

        return loss

    def validation_step(self, batch: torch.Tensor, batch_idx: int):
        x, y = batch
        output = self(x)
        loss = self.loss_fn(output, y)

        self._log_metrics(loss, output, y, "val")

        return loss

    def _log_metrics(
        self, loss: torch.Tensor, preds: torch.Tensor, target: torch.Tensor, phase: str
    ):
        accuracy = self.accuracy(preds, target)
        f1 = self.f1(preds, target)
        recall = self.recall(preds, target)
        precision = self.precision(preds, target)

        self.log(
            f"{phase}/loss",
            loss,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
            batch_size=self.batch_size,
        )
        self.log(
            f"{phase}/accuracy",
            accuracy,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
            batch_size=self.batch_size,
        )
        self.log(
            f"{phase}/f1",
            f1,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
            batch_size=self.batch_size,
        )
        self.log(
            f"{phase}/recall",
            recall,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
            batch_size=self.batch_size,
        )
        self.log(
            f"{phase}/precision",
            precision,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
            batch_size=self.batch_size,
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.model.parameters(), lr=self.lr)

        warmup = optim.lr_scheduler.LinearLR(optimizer, total_iters=self.warmup_epochs)
        exponential = optim.lr_scheduler.ExponentialLR(optimizer, gamma=self.gamma)
        scheduler = optim.lr_scheduler.SequentialLR(
            optimizer, schedulers=[warmup, exponential], milestones=[self.warmup_epochs]
        )

        return [optimizer], [scheduler]


In [5]:
# installing pyvips for offline use
# intall the deb packages
!yes | dpkg -i --force-depends /kaggle/input/pyvips-python-and-deb-package-gpu/linux_packages/archives/*.deb &> /dev/null
# install the python wrapper
!pip install pyvips -f /kaggle/input/pyvips-python-and-deb-package-gpu/python_packages/ --no-index &> /dev/null

yes: standard output: Broken pipe


In [6]:
# from https://www.kaggle.com/competitions/UBC-OCEAN/discussion/451908 &
# https://www.kaggle.com/code/jirkaborovec/cancer-subtype-lightning-torch-inference-tiles

import pyvips
import numpy as np
import random
from PIL import Image


# cuts tiles of specified size; filters tiles with more black pixels than drop_thr
def extract_image_tiles(
    p_img,
    folder,
    size: int = 2048,
    scale: float = 0.5,
    drop_thr: float = 0.6,
    white_thr: int = 240,
    max_samples: int = 50,
) -> list:
    name, _ = os.path.splitext(os.path.basename(p_img))
    im = pyvips.Image.new_from_file(p_img)
    w = size
    h = size
    
    # https://stackoverflow.com/a/47581978/4521646
    idxs = [
        (y, y + h, x, x + w)
        for y in range(0, im.height, h)
        for x in range(0, im.width, w)
    ]
    # random subsample
    max_samples = (
        max_samples if isinstance(max_samples, int) else int(len(idxs) * max_samples)
    )
    random.shuffle(idxs)
    files = []
    
    for y, y_, x, x_ in idxs:
        # https://libvips.github.io/pyvips/vimage.html#pyvips.Image.crop
        tile = im.crop(x, y, min(w, im.width - x), min(h, im.height - y)).numpy()[
            ..., :3
        ]
        
        # Extend edges if tile is smaller
        if tile.shape[:2] != (h, w):
            tile_ = tile
            tile_size = (h, w) if tile.ndim == 2 else (h, w, tile.shape[2])
            tile = np.zeros(tile_size, dtype=tile.dtype)
            tile[: tile_.shape[0], : tile_.shape[1], ...] = tile_
            
        black_bg = np.sum(tile, axis=2) == 0
        tile[black_bg, :] = 255
        mask_bg = np.mean(tile, axis=2) > white_thr
        
        if np.sum(mask_bg) >= (np.prod(mask_bg.shape) * drop_thr):
            continue
            
        p_img = os.path.join(folder, f"{int(x_ / w)}-{int(y_ / h)}.png")
        # print(tile.shape, tile.dtype, tile.min(), tile.max())
        new_size = int(size * scale), int(size * scale)
        Image.fromarray(tile).resize(new_size, Image.LANCZOS).save(p_img)
        files.append(p_img)
        # need to set counter check as some empty tiles could be skipped earlier
        if len(files) >= max_samples:
            break
    return files, idxs


# creates tiles for specified image /kaggle/input/UBC-OCEAN/test_images/*.png into folder /kaggle/working/test_tiles/*.png
def extract_prune_tiles(
    path_img: str,
    folder: str,
    size: int = 2048,
    scale: float = 0.25,
    drop_thr: float = 0.6,
    max_samples: int = 30,
) -> str:
    print(f"processing: {path_img}")
    name, _ = os.path.splitext(os.path.basename(path_img))
    folder = os.path.join(folder, name)
    os.makedirs(folder, exist_ok=True)
    tiles, _ = extract_image_tiles(
        path_img,
        folder,
        size=size,
        scale=scale,
        drop_thr=drop_thr,
        max_samples=max_samples,
    )
    print(f"found {len(tiles)} tiles")
    return folder


In [7]:
import os

# pyvips settings - important
os.environ['VIPS_CONCURRENCY'] = '4'
os.environ['VIPS_DISC_THRESHOLD'] = '15gb'

In [8]:
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import pandas as pd
import cv2
import glob
from PIL import Image

class CancerTilesDataset(Dataset):
    def __init__(
        self,
        folder: str,
        image_ext: str = '.png',
        preprocessing=None,
        augmentation=None,
    ):
        self.imgs = glob.glob(os.path.join(folder, "*" + image_ext))
        self.preprocessing = preprocessing
        self.augmentation = augmentation


    def __getitem__(self, i):
        image_id_list = self.imgs
        random.shuffle(image_id_list)
        
        suitable_imgs = []
        
        # Construct a bag
        for img_path in image_id_list:
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            if self.augmentation:
                img = self.augmentation(image=img)["image"]

            if self.preprocessing:
                img = self.preprocessing(image=img)["image"]

            suitable_imgs.append(img)
        
        bag = torch.stack(suitable_imgs)

        return bag

    def __len__(self):
        return 1


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
best_model = "/kaggle/input/cancer-detection-w-feature-level-aggregation/cancer_classification_model.pt"

# Load model from checkpoint
model = CancerDetector.load_from_checkpoint(best_model, init_weights=False)
model.to(device)
model.eval()

CancerDetector(
  (model): EfficientNetMIL(
    (feature_extractor1): EfficientNetFeatures(
      (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNormAct2d(
        48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inplace=True)
      )
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNormAct2d(
              48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): SiLU(inplace=True)
            )
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))


In [10]:
import gc

config = {
    "input_size": 256,
    "scale": 256 / 2048,
    "bag_size": 50
}

# Map index to class
classes = ['HGSC', 'LGSC', 'EC', 'CC', 'MC']
idx2class = {idx: class_name for idx, class_name in enumerate(classes)}

test_df = pd.read_csv("/kaggle/input/UBC-OCEAN/test.csv")

# Ajutiselt panen siia, parem integreerida klassi
aug_transforms = AugmentationTransforms(config["input_size"])

submission = []
import time
for _, row in test_df.iterrows():
    row = dict(row)

    # prepare data - cut and load tiles
    folder_tiles = extract_prune_tiles(
        os.path.join("/kaggle/input/UBC-OCEAN/", "test_images", f"{str(row['image_id'])}.png"),
        "./test_tiles/", size=2048, scale=config["scale"], max_samples = config["bag_size"])
    
    gc.collect()
    torch.cuda.empty_cache()
    
    
    with torch.no_grad():
        dataset = CancerTilesDataset(folder_tiles, preprocessing = aug_transforms.get_preprocessing())

        if not len(dataset):
            print (f"seem no tiles were cut for `{folder_tiles}`")
            submission.append(row)
            continue

        #dataloader = DataLoader(dataset, batch_size=4, num_workers=4, shuffle=False)

        imagePredictions = []
        maxScores = []  # List to store the maximum scores for each batch

        # Only one batch of size 1
        print("Predicting class")
        img_bag = dataset[0]

        img_bag = img_bag.to(device)
        output = model(img_bag)

        # Calculate the maximum softmax score for each prediction in the batch

        softmax_scores = torch.nn.functional.softmax(output, dim=1)
        max_score, preds = torch.max(softmax_scores, dim=1)
        maxScores.extend(max_score.detach().cpu().numpy())

        preds = preds.cpu().numpy()
        imagePredictions.append(preds)


        # Check if the average max score of predictions is below the threshold
    #     threshold = 0.8
    #     avgMaxScore = np.mean(maxScores)
    #     if avgMaxScore < threshold:
    #         row['label'] = 'Other'  # Classify as 'Other'
    #     else:
    #         # If above threshold, use the most frequent prediction
    #         lb = np.argmax(np.bincount(np.concatenate(imagePredictions)))
    #         row['label'] = classes[lb]

        row['label'] = classes[preds[0]]

    submission.append(row)
    
    # cleaning
    os.system(f"rm -rf {folder_tiles}")
    
    # Clean memory
    del dataset
    gc.collect()
    torch.cuda.empty_cache()

processing: /kaggle/input/UBC-OCEAN/test_images/41.png
found 50 tiles
Predicting class


In [11]:
df_sub = pd.DataFrame(submission)
df_sub[["image_id", "label"]].to_csv("submission.csv", index=False)